In [198]:
%run 01_hash.ipynb

In [197]:
from copy import deepcopy

Print objects in a nice way.

In [134]:
def stringfy(o):
    s = []
    for k,v in o.__dict__.items():
        p = f'{k}:'.ljust(15)
        if k =='time'        : s.append(p+time.ctime(v))
        elif str(v)[:2]=='0x': s.append(p+ph(v))
        elif type(v)   ==int : s.append(p+str(v)+' ether')
        else                 : s.append(p+str(v))
    return '\n'.join(s)

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [195]:
class TX: 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed']: super().__setattr__('hash', sha(self.__dict__))
        if prop == 'sig'               : self.signed = True
    
    def __str__(self):       return stringfy(self)
    def __bytes__(self):     return str(self.hash).encode()
    def __eq__(self, other): return self.hash == other.hash

Create random txs.

In [196]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0)

In [ ]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [189]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            🔀 0x9ad545390b...880
hash:          💄 0x1e71fb9d63...ee1
to:            👵 0x0f4a33b130...873
value:         120 ether
fee:           0.15
nonce:         0 ether
time:          Tue Mar 30 17:09:59 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [190]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [191]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            🔻 0xd5ff21e9ab...637
hash:          👯 0x0940168e79...eb4
to:            👶 0x1057c62878...57c
value:         12 ether
fee:           0.2
nonce:         0 ether
time:          Tue Mar 30 17:09:59 2021
signed:        True
sig:           🔏 0xa945de25a7...e1e


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [192]:
assert tx1 == tx1_signed